In [1]:
import os
import pandas as pd

from entity.LightNode import LightNode
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from entity import LightCluster, LightNode

path = ProjectPath()
setting = Setting()
dataloader = DataLoader()

cluster_path = path.univ2_cluster_path
cluster_file_template = "cluster_{}.csv"
suspicious_file_template = "cluster_{}_suspicious_nodes.csv"

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
LOAD END NODES
LOAD CREATION INFO


/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token0"] = tokens["token0"].str.lower()
/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token1"] = tokens["token1"].str.lower()


LOAD RUG PULL INFO


In [13]:
def count_hops_from_source(source_address, paths: list):
    source_address = source_address.lower()
    if not source_address in paths:
        return 0
    return len(paths[paths.index(source_address):])


def get_nodes_by_labels(root_label, target_labels, cluster: LightCluster):
    root_label_nodes = []
    target_label_nodes = dict()
    for node in cluster.nodes.values():
        if root_label in node.labels and not any(label in LightNode.LightNodeLabel.SKIP_LABELS for label in node.labels):
            root_label_nodes.append(node)
        for l in node.labels:
            if l in target_labels:
                nodes = []
                if l in target_label_nodes.keys():
                    nodes = target_label_nodes[l]
                nodes.append(node)
                target_label_nodes[l] = nodes
    return root_label_nodes, target_label_nodes


def find_impacts(root_node, target_nodes):
    satisfied_nodes = []
    hops = []
    for node in target_nodes:
        if root_node.address in node.path:
            satisfied_nodes.append(node)
            hops.append(count_hops_from_source(root_node.address, node.path))
    return satisfied_nodes, hops


def impact_analysis(root_label, target_labels, cluster: LightCluster):
    root_label_nodes, target_label_nodes = get_nodes_by_labels(root_label, target_labels, cluster)
    results = []
    for root in root_label_nodes:
        record = LightNode.LightNode.to_sort_dict(root)
        impacts = []
        for tl in target_label_nodes.keys():
            tgn = target_label_nodes.get(tl)
            satisfied_nodes, hops = find_impacts(root, tgn)
            impacts.append({"type": tl, "count": len(satisfied_nodes), "avg_distance": sum(hops) / len(hops) if len(hops) > 0 else 0})
        record["impacts"] = impacts
        results.append(record)
    return results




In [6]:
group_id = 3
cluster = LightCluster.LightCluster(group_id)
cluster.load(cluster_path)

LOAD EXISTING CLUSTER


In [14]:
results = impact_analysis(LightNode.LightNodeLabel.BIG_CONNECTOR, [LightNode.LightNodeLabel.BIG_CONNECTOR, LightNode.LightNodeLabel.SCAMMER, LightNode.LightNodeLabel.BIG_WASHTRADER], cluster)
display(results)

[{'address': '0x4e756eb534caf866f911117f9c5b9927c11927cf',
  'valid_neighbours': 152,
  'normal_txs_len': 438,
  'labels': 'withdrawer;depositor;big_connector',
  'impacts': [{'type': 'scammer', 'count': 0, 'avg_distance': 0},
   {'type': 'big_washtrader', 'count': 48, 'avg_distance': 2.0},
   {'type': 'big_connector', 'count': 9, 'avg_distance': 1.8888888888888888}]},
 {'address': '0x167c6a6948606e410cd79baac3be52d949cbcd3c',
  'valid_neighbours': 135,
  'normal_txs_len': 2364,
  'labels': 'big_washtrader;over_limit_2_but_accept;depositor;withdrawer;big_connector',
  'impacts': [{'type': 'scammer', 'count': 127, 'avg_distance': 2.0},
   {'type': 'big_washtrader', 'count': 8, 'avg_distance': 1.875},
   {'type': 'big_connector', 'count': 2, 'avg_distance': 1.5}]},
 {'address': '0xfa19e5e2ca003c0195859b10296867a2a10514f9',
  'valid_neighbours': 58,
  'normal_txs_len': 698,
  'labels': 'big_washtrader;over_limit_2_but_accept;big_connector',
  'impacts': [{'type': 'scammer', 'count': 41, '

In [16]:
df = pd.read_csv(os.path.join(path.panv2_processed_path, "1_pair_pool_labels.csv"))
display(df.head())

,pool,creator,is_rp,token0,token1,scam_token
0,0x804678fa97d91B974ec2af3c843270886528a9E6,0xfca08fd2057a995cd270f22076c902b5cd2b4237,0,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,0xe9e7cea3dedca5984780bafc599bd69add087d56,NaN
1,0x0eD7e52944161450477ee417DE9Cd3a859b14fD0,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN
2,0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe9e7cea3dedca5984780bafc599bd69add087d56,NaN
3,0x28415ff2C35b65B9E5c7de82126b4015ab9d031F,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0x3ee2200efb3400fabb9aacf31297cbdd1d435d47,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN
4,0x168B273278F3A8d302De5E879aA30690B7E6c28f,0x36469e4e6ec2fb3e06db1d747f3fc1f1fc762f02,0,0xad6caeb32cd2c308980a548bd0bc5aa4306c6c18,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,NaN


In [22]:
rp = df[df["is_rp"] == 2]
rp

,pool,creator,is_rp,token0,token1,scam_token
341,0x3cF8d73a4C3542Add0eE79cd3DE4B9DC4BB14218,0x2a218d74c215c5eaee4ef6e12740c9bfed029cc0,2,0x02be07f9f2b983843cb49e3819903dcef673fbe6,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x02be07f9f2b983843cb49e3819903dcef673fbe6
448,0xb1c29d750e7095564bCd00cBBaa66a5166620eFe,0xbc335e8c9ebb7ca2ab2a9e8dca57ff5122405212,2,0x2bba875facd913a89bbd7e4607854dcbe03db721,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x2bba875facd913a89bbd7e4607854dcbe03db721
530,0x3C2DA1C1e7db3d138D5345de824a547272bC258B,0xa9484a784288f9f3d03abc2cfa4387ff52cacbe0,2,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe3d94973ab0056218dc9312bc0a617f61a411c57,0xe3d94973ab0056218dc9312bc0a617f61a411c57
921,0xcaD115d96819d0f4cd8D2856f81bA48d12FF3cf1,0xeb4b9ce2fc2d9998ca72c54e00b979ea92bf4832,2,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xec88b8e96e842de86c0327dc5cdd4b04dc69ecd7,0xec88b8e96e842de86c0327dc5cdd4b04dc69ecd7
934,0xAf4aEa085E44499AbBd1e1Be9A5555DfE2A52C7d,0xbb1f9c860dbd57f4cb81cfb538b2a0dfba24fcb6,2,0x953dcb0ce0d366501ea3748709a5acd6608c3a95,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x953dcb0ce0d366501ea3748709a5acd6608c3a95
...,...,...,...,...,...,...
1381368,0xB3dce8737600b146700EEC7e9b40a699FF92Cd96,0x1f40c3039c2fb107a7732bef35361a7336a42ad7,2,0x9427fcbd282beca75a2a3a415bdac1f192dc64cd,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x9427fcbd282beca75a2a3a415bdac1f192dc64cd
1381376,0x1d5329E7De0152AB0E1c3e00A1DD9CCd73138E55,0xd70e3a874e8ae3d0efaad47130a337cdf998d2ce,2,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xf3a21969c9582567241759d1c1a04626a98ed3ca,0xf3a21969c9582567241759d1c1a04626a98ed3ca
1381406,0xBB26356FB011acE9Cd9504521793eAFaE17C17eE,0xf9ce89a8d6e38ac332c136237b085b2367885342,2,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe2f833ddc9e0ea930ffd9ed93f67a191621836ab,0xe2f833ddc9e0ea930ffd9ed93f67a191621836ab
1381412,0x8a074594b0d5eF825D75cE115360df13f4F3f51B,0x8f137c73f635226dedb6373addff1bfa4b7171f5,2,0x90c2c79ac7d198f2b4f09e66c387b7c63f6275c7,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x90c2c79ac7d198f2b4f09e66c387b7c63f6275c7
